# 09 Figures for Manuscript
creating the official figure .png files for submission

In [1]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import numpy as np;
import pandas as pd;
import plotnine as p9;
import pickle

import warnings
warnings.filterwarnings( "ignore", module = "plotnine\..*" ) # only include this once finished and want to create nicer figures for pdf


C:\Users\Lauren.Koenig\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from PIL import Image
import io

def plotnine_to_pil_png(plot):
    """Converts a plotnine plot to a PIL Image object (PNG format).

    Args:
      plot: A plotnine ggplot object.

    Returns:
      A PIL Image object (PNG format) representing the plot.
    """

    buf = io.BytesIO()
    plot.save(buf, format='png')  # Save the plot to the buffer in PNG format
    buf.seek(0)  # Move the buffer's cursor to the beginning
    img = Image.open(buf)  # Open the image from the buffer using PIL
    return img

def combine_images(columns, space, file_loc, images):
    rows = len(images) // columns
    if len(images) % columns:
        rows += 1
    width_max = max([image.width for image in images])
    height_max = max([image.height for image in images])
    background_width = width_max*columns + (space*columns)-space
    background_height = height_max*rows + (space*rows)-space
    background = Image.new('RGBA', (background_width, background_height), (255, 255, 255, 255))
    x = 0
    y = 0
    for i, image in enumerate(images):
        img = image
        x_offset = int((width_max-img.width)/2)
        y_offset = int((height_max-img.height)/2)
        background.paste(img, (x+x_offset, y+y_offset))
        x += width_max + space
        if (i+1) % columns == 0:
            y += height_max + space
            x = 0
    background.save(file_loc )

# read in and clean data

In [3]:
data_folder_loc = '//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/'
code_folder_loc = '//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/Riluzole FDG/'

biomarker_relationship_results_log10, biomarker_relationship_results_raw, all_data = pd.read_pickle(code_folder_loc + '/output/05a_data_new_para_rr.pkl')
all_data = all_data[all_data['timepoint'].isin(['base'])]

all_data = all_data.rename(columns = {'race':'race_ethnicity'})

full_names = pd.read_csv(code_folder_loc + 'data/data-dictionary.csv')

In [4]:
biomarker_relationship_results_log10[biomarker_relationship_results_log10['y_var'].isin([

'VOL_InfInsFrontal_Lz', 
'VOL_InfInsFrontal_Rz', 


'VOL_Precun_InfPar_Lz', 
'VOL_Precun_InfPar_Rz', 
'VOL_Precun_InfPar_Supramarg_Lz', 
'VOL_Precun_InfPar_Supramarg_Rz', 
'VOL_InfParSupra_Lz', 
'VOL_InfParSupra_Rz' ,


'VOL_LatOccLingCun_Lz', 
'VOL_LatOccLingCun_Rz', 
]) & 
(biomarker_relationship_results_log10['model_p_val_x_var']<0.05) & 
~(biomarker_relationship_results_log10['x_var'].isin(['pTau181_Ab42', 'Ab42']))][['y_var', 'x_var', 'model_p_val_x_var']].round(3).sort_values('x_var')

,y_var,x_var,model_p_val_x_var
0,VOL_InfParSupra_Lz,GFAP,0.038
0,VOL_Precun_InfPar_Lz,pTau217,0.028
0,VOL_Precun_InfPar_Supramarg_Lz,pTau217,0.009
0,VOL_Precun_InfPar_Supramarg_Rz,pTau217,0.036
0,VOL_InfParSupra_Lz,pTau217,0.005
0,VOL_InfParSupra_Rz,pTau217,0.035
0,VOL_InfInsFrontal_Rz,pTau231,0.002
0,VOL_Precun_InfPar_Rz,pTau231,0.012
0,VOL_Precun_InfPar_Supramarg_Rz,pTau231,0.009
0,VOL_LatOccLingCun_Lz,pTau231,0.040


In [5]:
len(all_data)

51

In [6]:
full_names.loc[full_names['variable_name'].isin(['VOL_InfParSupra_Lz']), 'full_name'] 

109    Left Inferior Parietal Volume z-score
Name: full_name, dtype: object

In [7]:
# add line breaks to long names as needed
#full_names.loc[full_names['variable_name'].isin(['VOL_InfParSupra_Lz']), 'full_name'] = 'Left Inferior Parietal Volume\nz-score'
full_names.loc[full_names['variable_name'].isin(['VOL_ParaHip_Lz']), 'full_name'] = 'Left Parahippocampal\Volume z-score'


In [8]:
# define biomarker groups
FDG_columns = ['MTL_gm', 'SensMot_gm', 'FRONTAL_gm', 'AC_gm', 'Precun_gm', 'Par_gm', 'Temp_gm', 'Paracentral_gm', 'PostCing_gm']

plasma_columns = ['Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau217_Ab42']
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

cog_columns = ['MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']
vol_columns = ['VOL_Precun_Lz', 'VOL_Precun_Rz', 'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz',
 'VOL_Inf_Mid_Fus_Temp_Lz', 'VOL_Inf_Mid_Fus_Temp_Rz', 'VOL_MidFront_Rz', 'VOL_MidFront_Lz']

all_data['VOL_Hip_z'] = all_data[['VOL_Hip_Rz', 'VOL_Hip_Lz']].mean(axis = 1)

graphing_pval = 0.05
pvals_to_consider = ['model_p_val_x_var']
Bval_to_grab = 'model_B_val_x_var'


all_data['age_decade2'] = pd.cut(all_data['age'], [57, 70, 77, 88]).astype('str').str.replace('nan', 'unknown')

all_data['MMSE_bin'] = pd.cut(all_data['MMSE'], [18, 20, 23, 27]).astype('str').str.replace('nan', 'unknown')
all_data['cdrsum_bin'] = pd.cut(all_data['cdrsum'], [0, 2.5, 3.5, 10]).astype('str').str.replace('nan', 'unknown')
all_data['Ab42_40_bin'] = pd.cut(all_data['Ab42_40'], [0.04, 0.06, 0.07,  0.11]).astype('str').str.replace('nan', 'unknown')
all_data['pTau217_bin'] = pd.cut(all_data['pTau217'], [0.2, 1.1, 2, 4.5]).astype('str').str.replace('nan',  'unknown')
all_data['GFAP_bin'] = pd.cut(all_data['GFAP'], [75, 175, 275, 475]).astype('str').str.replace('nan', 'unknown')
all_data['NFL_bin'] = pd.cut(all_data['NFL'], [15, 30, 45, 90]).astype('str').str.replace('nan', 'unknown')
all_data['VOL_Hip_z_bin'] = pd.cut(all_data['VOL_Hip_z'], [-5.4, -2.7, -1.2, 1.3]).astype('str').str.replace('nan', 'unknown')

In [9]:
all_data = all_data[all_data['age'].notna()]

In [10]:
def pval_rounder(pval):
    if (pval < 0.001):
        pval_str = 'p<0.001'
    elif (pval < 0.05) & (pval >= 0.04995):
        pval_str = 'p=' + str(round(pval, 5))
    elif (pval < 0.05) & (pval >= 0.0495):
        pval_str = 'p=' + str(round(pval, 4))
    elif (pval < 0.05) & (pval >= 0.045):
        pval_str = 'p=' + str(round(pval, 3))
    elif (pval < 0.01) & (pval > 0.001):
        pval_str = 'p=' + str(round(pval, 3))
    elif (pval >= 0.01):
        pval_str = 'p=' + str(round(pval, 2))
    return pval_str



def real_name(var):
    if var in full_names['variable_name'].to_list():
        var_str = full_names.loc[full_names['variable_name'].isin([var]), 'full_name'].to_list()[0]
    elif (var == 'Paracentral_gm') : var_str = 'Paracentral Cortex FDG' 
    elif (var == 'Ab42_40_bin') : var_str = 'Plasma A' + '\u03B2' + '\n42/40 Group\n' 
    elif (var == 'pTau217_bin') : var_str = 'Plasma p-tau\n217 Group\n' 
    elif (var == 'GFAP_bin') : var_str = 'Plasma\nGFAP Group\n' 
    elif (var == 'NFL_bin') : var_str = 'Plasma\nNFL Group\n' 
    elif (var == 'MMSE_bin') : var_str = 'MMSE Group' 
    elif (var == 'cdrsum_bin') : var_str = 'CDR Group' 
    elif (var == 'VOL_Hip_z_bin') : var_str = 'Average\nHippocampal\nz-score Group\n\n' 
    elif (var == 'age') : var_str = 'Age' 
    elif (var == 'Education_years') : var_str = 'Years of Education' 
    elif (var == 'sex') : var_str = 'Sex' 
    elif (var == 'apoe4_carrier') : var_str = 'APOE E4 Carrier' 
    elif (var == 'race_ethnicity') : var_str = 'Race and Ethnicity' 
    elif (var == 'age_decade2') : var_str = 'Age Group' 
    else:
        var_str = var
    if (var_str == var):
        raise Exception('missing full name')
    return var_str

In [11]:
all_data[all_data[FDG_columns + vol_columns + cog_columns + plasma_columns].isna().all(axis = 1) & all_data[plasma_columns].isna().all(axis = 1)].dropna(axis = 1, how = 'all')

,Subject_Label,Subject_Number,ID_ADM,timepoint,age,sex,Diagnosis,age_decade,age_decade2,source,missing_vars,missing_vars2,MMSE_bin,cdrsum_bin,Ab42_40_bin,pTau217_bin,GFAP_bin,NFL_bin,VOL_Hip_z_bin
137,APE-792,55,APE-792_55,base,81.0,M,AD,"(80, 90]","(77, 88]",Riluzole Trial Participants,362,21,unknown,unknown,unknown,unknown,unknown,unknown,unknown


In [12]:
all_data = all_data[all_data[FDG_columns + vol_columns + cog_columns + plasma_columns].notna().any(axis = 1) & all_data[plasma_columns].notna().any(axis = 1)]

In [58]:
len(all_data[all_data[FDG_columns + vol_columns + cog_columns + plasma_columns].notna().any(axis = 1) & all_data[plasma_columns].notna().any(axis = 1)])

42

In [59]:
all_data[all_data[FDG_columns + vol_columns + cog_columns + plasma_columns].notna().any(axis = 1) & all_data[plasma_columns].notna().any(axis = 1)][['age', 'Education_years', 'sex', 'apoe4_carrier', 'race_ethnicity']].isna().value_counts()

age    Education_years  sex    apoe4_carrier  race_ethnicity
False  False            False  False          False             37
                               True           True               4
                                              False              1
Name: count, dtype: int64

In [60]:
all_data[FDG_columns + vol_columns + cog_columns + plasma_columns].notna().sum()

MTL_gm                     42
SensMot_gm                 42
FRONTAL_gm                 42
AC_gm                      42
Precun_gm                  42
Par_gm                     42
Temp_gm                    42
Paracentral_gm             42
PostCing_gm                42
VOL_Precun_Lz              37
VOL_Precun_Rz              37
VOL_InfPar_Lz              37
VOL_InfPar_Rz              37
VOL_Hip_Lz                 37
VOL_Hip_Rz                 37
VOL_TotalGrayz             37
VOL_Inf_Mid_Fus_Temp_Lz    37
VOL_Inf_Mid_Fus_Temp_Rz    37
VOL_MidFront_Rz            37
VOL_MidFront_Lz            37
MMSE                       38
adascogtotal               38
bvrt                       37
dstotal                    38
tma                        35
tmb                        19
cowattotal                 38
cdrtotal                   38
cdrsum                     38
adltotal                   38
npitotal                   38
gds                        26
Ab42_40                    35
GFAP      

# Table 1 - Demographics

In [77]:
for var in ['age', 'Education_years', 'MMSE']:
    #print(var, '\n', all_data[var].describe().round(1), '\n\n')
    print(var + ' mean (std) = ' + all_data[var].mean().round(1).astype('str') + ' (S.D. ' + all_data[var].std().round(1).astype('str') + ')')
for var in ['sex', 'race_ethnicity', 'apoe4_carrier']:
    print(var, '\n', all_data[var].value_counts().sort_index(), '\n',
     (all_data[var].value_counts(normalize=True).sort_index().mul(100).round().astype('int').astype(str) + '%'), '\n\n')
for var in plasma_columns:
    print(var + ' mean (std) = ' + all_data[var].mean().round(3).astype('str') + ' (S.D. ' + all_data[var].std().round(3).astype('str') + ')')
#for var in [ 'Par_gm', 'Temp_gm']:
    #print(var + ' mean (std) = ' + all_data[var].mean().round(2).astype('str') + ' (' + all_data[var].std().round(2).astype('str') + \
    #     '). Range = '+ all_data[var].min().round(2).astype('str') + '-' + all_data[var].max().round(2).astype('str'))
    #print( all_data[var].describe().round(1), '\n')

age mean (std) = 74.6 (S.D. 7.0)
Education_years mean (std) = 15.5 (S.D. 2.9)
MMSE mean (std) = 22.6 (S.D. 2.8)
sex 
 sex
F    26
M    16
Name: count, dtype: int64 
 sex
F    62%
M    38%
Name: proportion, dtype: object 


race_ethnicity 
 race_ethnicity
Black        2
White/NH    36
Name: count, dtype: int64 
 race_ethnicity
Black        5%
White/NH    95%
Name: proportion, dtype: object 


apoe4_carrier 
 apoe4_carrier
No     15
Yes    22
Name: count, dtype: int64 
 apoe4_carrier
No     41%
Yes    59%
Name: proportion, dtype: object 


Ab42_40 mean (std) = 0.065 (S.D. 0.012)
GFAP mean (std) = 232.344 (S.D. 96.338)
NFL mean (std) = 37.041 (S.D. 15.138)
pTau181 mean (std) = 9.526 (S.D. 5.333)
pTau217 mean (std) = 1.626 (S.D. 0.785)
pTau231 mean (std) = 26.986 (S.D. 19.068)
pTau217_Ab42 mean (std) = 0.396 (S.D. 0.236)


In [67]:
print('any missing covariate data?')
all_data[['age', 'Education_years', 'sex', 'race_ethnicity', 'apoe4_carrier']].isna().value_counts().sort_index()

any missing covariate data?


age    Education_years  sex    race_ethnicity  apoe4_carrier
False  False            False  False           False            37
                                               True              1
                               True            True              4
Name: count, dtype: int64

# Table 2, 3, 4, and S1 extracted from supplemental table

# Figure 1 - Ptau-217 results
inferior parietal PET, lateral temporal PET, precuneus PET, VOL_InfParSupra_Lz, Benton Retention Test, Trails A, 


In [68]:
p9.options.figure_size = (4,3)    
color_group = 'age_decade2'

x_var = 'pTau217'
y_var = 'Par_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1a =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('a. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'Temp_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1b =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('b. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'Precun_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1c =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('c. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'VOL_InfParSupra_Lz'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1d =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('d. ' + pval_rounder(p_value) + ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'bvrt'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1e =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('e. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

x_var = 'pTau217'
y_var = 'tma'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1f =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('f. ' + pval_rounder(p_value) + ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )




fig_1a_png = plotnine_to_pil_png(fig_1a)
fig_1b_png = plotnine_to_pil_png(fig_1b)
fig_1c_png = plotnine_to_pil_png(fig_1c)
fig_1d_png = plotnine_to_pil_png(fig_1d)
fig_1e_png = plotnine_to_pil_png(fig_1e)
fig_1f_png = plotnine_to_pil_png(fig_1f)

combine_images(columns=2, space=20, file_loc = code_folder_loc + 'output/figure1.png', \
    images=[fig_1a_png, fig_1b_png, fig_1c_png, fig_1d_png, fig_1e_png, fig_1f_png])

# Figure 2  Voxel-level relationship between p-tau 217 and FDG PET 
provided by Dawn

# Figure 3 - GFAP Results

In [69]:
x_var = 'GFAP'
y_var = 'FRONTAL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3a =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('a. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new b
x_var = 'GFAP'
y_var = 'NFL'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3b =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('b. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new c
x_var = 'GFAP'
y_var = 'pTau217'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3c =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('c. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new d
x_var = 'GFAP'
y_var = 'dstotal'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3d =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('d. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new d
x_var = 'GFAP'
y_var = 'tma'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')

fig_3e =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('e. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

x_var = 'GFAP'
y_var = 'VOL_InfParSupra_Lz'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3f =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('f. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


fig_3a_png = plotnine_to_pil_png(fig_3a)
fig_3b_png = plotnine_to_pil_png(fig_3b)
fig_3c_png = plotnine_to_pil_png(fig_3c)
fig_3d_png = plotnine_to_pil_png(fig_3d)
fig_3e_png = plotnine_to_pil_png(fig_3e)
fig_3f_png = plotnine_to_pil_png(fig_3f)

combine_images(columns=2, space=20, file_loc = code_folder_loc + 'output/figure3.png', \
    images=[fig_3a_png, fig_3b_png, fig_3c_png, fig_3d_png, fig_3e_png, fig_3f_png])

# Figure S2 - Bootstrapping

In [70]:
# import required packages
import numpy as np;
import pandas as pd;
from sklearn import preprocessing 
from sklearn.utils import resample
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import itertools
import pickle

import scipy; # for spearmann correlation

def bootstrap_regression(original_result, x_var, y_var):
    biomarker_relationship_results_log10 = pd.DataFrame()
    biomarker_relationship_results_log10_975 = pd.DataFrame()
    biomarker_relationship_results_log10_025 = pd.DataFrame()

    covariate_p_threshold = 0.05

        
    temp_df_baseline =  all_data.copy().dropna(subset = [y_var, x_var])

    # z-score data
    temp_df_baseline[[y_var, x_var, 'Education_years', 'age']] = preprocessing.StandardScaler().fit_transform(temp_df_baseline[[y_var, x_var, 'Education_years', 'age']])

    # run 1 - check for significant covariates
    sig_pvals_start = ['age', 'Education_years', 'apoe4_carrier', 'sex', 'race_ethnicity', x_var]
    change = 1
    full_model = smf.ols(formula = y_var + ' ~ ' + ' + '.join(sig_pvals_start), data = temp_df_baseline).fit()
    temp_df_baseline['full_model_residuals'] = full_model.resid

    while change > 0:
        model_vars = ' + '.join(sig_pvals_start)        
        test_model = smf.ols(formula = y_var + ' ~ ' + model_vars, data = temp_df_baseline).fit()
        sig_pvals = test_model.pvalues[test_model.pvalues < covariate_p_threshold].index.to_list()
        sig_pvals = [ele.split('[')[0] for ele in sig_pvals] # drop the [] indicators
        sig_pvals_unique = []
        for item in sig_pvals:
            if (item not in sig_pvals_unique) & (item not in ['Intercept']): sig_pvals_unique.append(item)
        if (x_var not in sig_pvals_unique):
            sig_pvals_unique = sig_pvals_unique + [x_var]
        change = len(sig_pvals_start) - len(sig_pvals_unique)
        sig_pvals_start = sig_pvals_unique
    
    # run 2 -with just the significant covariates (but forcing the x_var to be included)
    model_vars = ' + '.join(sig_pvals_unique)  

    resampled_df = pd.DataFrame()
    for resample_idx in range(0,1000):
        # multivariate linear model with age and education years as covariates
        resampled_data = resample(temp_df_baseline, replace=True, n_samples=None, random_state=resample_idx, stratify=None)
        model = smf.ols(formula = y_var + ' ~ ' + model_vars , data = resampled_data).fit()
        resampled_result = pd.DataFrame([model.pvalues, model.params])
        resampled_result['index'] = ['p', 'B']
        resampled_result.index = [0,0]
        resampled_result = resampled_result.pivot(columns = ['index'])
        resampled_result.columns = [f'{y}_{x}' for x,y in resampled_result.columns]
        resampled_result
        resampled_df = pd.concat([resampled_df, resampled_result])

    model_n_baseline = len(temp_df_baseline.dropna(subset = sig_pvals_unique + [y_var]))


    p9.options.figure_size = (4,3)    
    title = real_name(y_var) 
    subtitle = 'original ' + '\u03B2' + '=' + format(original_result.loc[original_result['x_var'].isin([x_var.replace('_log10', '')]) & original_result['y_var'].isin([y_var.replace('_log10', '')]),'model_B_val_x_var'].to_list()[0], '.2f') \
        + '\nbootstrap ' + '\u03B2' + '=' + format(resampled_df['B_' + x_var].quantile(q=0.5), '.2f') + ' (' + format(resampled_df['B_' + x_var].quantile(q=0.025), '.2f') + ' to ' + format(resampled_df['B_' + x_var].quantile(q=0.975), '.2f') + ')'
    plot1 = (
            p9.ggplot(resampled_df, p9.aes(x = 'B_' + x_var))
            + p9.theme_bw(base_size = 11)            
            + p9.geom_histogram(size = 0.5, bins = 20)
            + p9.geom_vline(xintercept = resampled_df['B_' + x_var].quantile(q=0.025), color = 'red')
            + p9.geom_vline(xintercept = resampled_df['B_' + x_var].quantile(q=0.975), color = 'red')
            + p9.geom_vline(xintercept = resampled_df['B_' + x_var].quantile(q=0.5), color = 'blue')   
            + p9.labs(title = title, subtitle = subtitle , x= '\u03B2' + '-value for ' + real_name(x_var))   
            + p9.scale_x_continuous(breaks = np.arange(round(resampled_df['B_' + x_var].min(),1), round(resampled_df['B_' + x_var].max(), 1), 0.2).round(1))
        )
    title = real_name(y_var) 
    subtitle =  'original ' + str(pval_rounder(original_result.loc[original_result['x_var'].isin([x_var.replace('_log10', '')]) & original_result['y_var'].isin([y_var.replace('_log10', '')]), 'model_p_val_x_var'].to_list()[0])) \
        + '\nbootstrap ' + pval_rounder(resampled_df['p_' + x_var].quantile(q=0.5)) + ' (' + pval_rounder(resampled_df['p_' + x_var].quantile(q=0.025)).replace('p=', '') + ' to ' + pval_rounder(resampled_df['p_' + x_var].quantile(q=0.975)).replace('p=', '') + ')'
    plot2 = (
            p9.ggplot(resampled_df, p9.aes(x = 'p_' + x_var))
            + p9.theme_bw(base_size = 11)            
            + p9.geom_histogram(size = 0.5, bins = 20)
            + p9.geom_vline(xintercept = resampled_df['p_' + x_var].quantile(q=0.025), color = 'red')
            + p9.geom_vline(xintercept = resampled_df['p_' + x_var].quantile(q=0.975), color = 'red')
            + p9.geom_vline(xintercept = resampled_df['p_' + x_var].quantile(q=0.5), color = 'blue')   
            + p9.labs(title = title, subtitle = subtitle, x= 'p-value for ' + real_name(x_var))   
            + p9.scale_x_continuous(breaks = np.arange(round(resampled_df['p_' + x_var].min(),2), round(resampled_df['p_' + x_var].max(), 2), 0.1).round(2))
            )

            
    return plot1,  plot2

In [71]:
fig_s3a,  fig_s3b  = bootstrap_regression(biomarker_relationship_results_log10, 'pTau217_log10', 'Par_gm')
fig_s3c,  fig_s3d  = bootstrap_regression(biomarker_relationship_results_log10, 'pTau217_log10', 'Temp_gm')
fig_s3e,  fig_s3f  = bootstrap_regression(biomarker_relationship_results_log10, 'pTau217_log10', 'SensMot_gm')
fig_s3g,  fig_s3h  = bootstrap_regression(biomarker_relationship_results_log10, 'pTau217_log10', 'Precun_gm')
fig_s3i,  fig_s3j  = bootstrap_regression(biomarker_relationship_results_log10, 'pTau217_log10', 'pTau231_log10')
fig_s3k,  fig_s3l  = bootstrap_regression(biomarker_relationship_results_log10, 'pTau181_log10', 'pTau231_log10')
fig_s3m,  fig_s3n  = bootstrap_regression(biomarker_relationship_results_log10, 'pTau181_log10', 'pTau217_log10')

fig_s3a_png = plotnine_to_pil_png(fig_s3a)
fig_s3b_png = plotnine_to_pil_png(fig_s3b)
fig_s3c_png = plotnine_to_pil_png(fig_s3c)
fig_s3d_png = plotnine_to_pil_png(fig_s3d)
fig_s3e_png = plotnine_to_pil_png(fig_s3e)
fig_s3f_png = plotnine_to_pil_png(fig_s3f)
fig_s3g_png = plotnine_to_pil_png(fig_s3g)
fig_s3h_png = plotnine_to_pil_png(fig_s3h)
fig_s3i_png = plotnine_to_pil_png(fig_s3i)
fig_s3j_png = plotnine_to_pil_png(fig_s3j)
fig_s3k_png = plotnine_to_pil_png(fig_s3k)
fig_s3l_png = plotnine_to_pil_png(fig_s3l)
fig_s3m_png = plotnine_to_pil_png(fig_s3m)
fig_s3n_png = plotnine_to_pil_png(fig_s3n)

#combine_images(columns=2, space=20, file_loc = code_folder_loc + 'output/figureS2.png', \
#    images=[fig_s3a_png, fig_s3b_png, fig_s3c_png, fig_s3d_png, fig_s3e_png, fig_s3f_png, fig_s3g_png, fig_s3h_png, fig_s3i_png, fig_s3j_png, fig_s3k_png, fig_s3l_png, fig_s3m_png, fig_s3n_png])

combine_images(columns=4, space=20, file_loc = code_folder_loc + 'output/figureS2.png', \
    images=[fig_s3a_png, fig_s3b_png, fig_s3c_png, fig_s3d_png, fig_s3e_png, fig_s3f_png, fig_s3g_png, fig_s3h_png, fig_s3i_png, fig_s3j_png, fig_s3k_png, fig_s3l_png, fig_s3m_png, fig_s3n_png])

# Figure S3: P-tau Interrelationships 

In [72]:
x_var = 'pTau181'
y_var = 'pTau217'


p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s1a =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('a. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau231'
y_var = 'pTau217'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s1b =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')            
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('b. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau181'
y_var = 'pTau231'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s1c =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')            
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('c. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

fig_s1a_png = plotnine_to_pil_png(fig_s1a)
fig_s1b_png = plotnine_to_pil_png(fig_s1b)
fig_s1c_png = plotnine_to_pil_png(fig_s1c)

combine_images(columns=2, space=20, file_loc = code_folder_loc + 'output/figureS3.png', \
    images=[fig_s1a_png, fig_s1b_png, fig_s1c_png])

# Figure S4: Medial Temporal Lobe FDG & p-tau 181 and p-tau 231 relationship persists in subgroups

In [73]:
p9.options.figure_size = (4,3)    
color_group = 'age_decade2'

x_var = 'pTau181'
y_var = 'MTL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s2a =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('a. Overall Relationship'), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

x_var = 'pTau231'
y_var = 'MTL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s2b =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            #+ p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('b. Overall Relationship in P-tau 231'), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

color_group = 'age_decade2'
x_var = 'pTau181'
y_var = 'MTL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s2c =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('c. Split By Age'), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

color_group = 'cdrsum_bin'
x_var = 'pTau181'
y_var = 'MTL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s2d =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('d. Split By CDR SOB'), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#024c02', '#2185e0', '#ac5bc5' , '#737373'])
        )


color_group = 'Ab42_40_bin'
x_var = 'pTau181'
y_var = 'MTL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s2e =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('e. Split By Plasma A' + '\u03B2' + ' 42/40' ), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_brewer(type='qual', palette='Set1')
        )

color_group = 'GFAP_bin'
x_var = 'pTau181'
y_var = 'MTL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_s2f =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('f. Split By Plasma GFAP'), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['darkgreen', 'darkturquoise', 'cornflowerblue', 'grey'])
        )


fig_s2a_png = plotnine_to_pil_png(fig_s2a)
fig_s2b_png = plotnine_to_pil_png(fig_s2b)
fig_s2c_png = plotnine_to_pil_png(fig_s2c)
fig_s2d_png = plotnine_to_pil_png(fig_s2d)
fig_s2e_png = plotnine_to_pil_png(fig_s2e)
fig_s2f_png = plotnine_to_pil_png(fig_s2f)

combine_images(columns=2, space=20, file_loc = code_folder_loc + 'output/figureS4.png', \
    images=[fig_s2a_png, fig_s2b_png, fig_s2c_png, fig_s2d_png, fig_s2e_png, fig_s2f_png])

# Supplement with age group line

In [74]:
p9.options.figure_size = (4,3)    
color_group = 'age_decade2'

x_var = 'pTau217'
y_var = 'Par_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1a =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('a. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'Temp_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1b =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('b. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'Precun_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1c =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('c. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'VOL_InfParSupra_Lz'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1d =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('d. ' + pval_rounder(p_value) + ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


x_var = 'pTau217'
y_var = 'bvrt'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1e =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('e. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

x_var = 'pTau217'
y_var = 'tma'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_1f =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('f. ' + pval_rounder(p_value) + ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )




fig_1a_png = plotnine_to_pil_png(fig_1a)
fig_1b_png = plotnine_to_pil_png(fig_1b)
fig_1c_png = plotnine_to_pil_png(fig_1c)
fig_1d_png = plotnine_to_pil_png(fig_1d)
fig_1e_png = plotnine_to_pil_png(fig_1e)
fig_1f_png = plotnine_to_pil_png(fig_1f)

combine_images(columns=2, space=20, file_loc = code_folder_loc + 'output/figure1_w_age_lines.png', \
    images=[fig_1a_png, fig_1b_png, fig_1c_png, fig_1d_png, fig_1e_png, fig_1f_png])

In [75]:
x_var = 'GFAP'
y_var = 'FRONTAL_gm'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3a =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('a. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new b
x_var = 'GFAP'
y_var = 'NFL'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3b =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('b. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new c
x_var = 'GFAP'
y_var = 'pTau217'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3c =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('c. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new d
x_var = 'GFAP'
y_var = 'dstotal'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3d =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('d. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

# new d
x_var = 'GFAP'
y_var = 'tma'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')

fig_3e =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('e. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )

x_var = 'GFAP'
y_var = 'VOL_InfParSupra_Lz'
p_value = biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), pvals_to_consider[0]].reset_index(drop=True)[0]
B_value = format(biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['x_var'].isin([x_var]) & biomarker_relationship_results_log10['y_var'].isin([y_var]), Bval_to_grab].reset_index(drop=True)[0], '.2f')
fig_3f =  (
            p9.ggplot(all_data[all_data[[color_group, x_var, y_var]].notna().all(axis = 1)], p9.aes(x = x_var, y = y_var, color = color_group))
            + p9.theme_bw(base_size = 11)
            + p9.geom_smooth(size = 1, se = False, alpha = 1, method = 'lm')                
            + p9.geom_smooth(size = 1.5, se = False, alpha = 1, color = 'black', method = 'lm')           
            + p9.geom_point(size = 1.0)
            + p9.labs(title = ('f. ' + pval_rounder(p_value)+ ' and ' + '\u03B2' + '=' + B_value), x=real_name(x_var), y=real_name(y_var), color=real_name(color_group))
            + p9.scale_color_manual(['#1b9e77', '#d95f02', '#7570b3', '#737373'])
        )


fig_3a_png = plotnine_to_pil_png(fig_3a)
fig_3b_png = plotnine_to_pil_png(fig_3b)
fig_3c_png = plotnine_to_pil_png(fig_3c)
fig_3d_png = plotnine_to_pil_png(fig_3d)
fig_3e_png = plotnine_to_pil_png(fig_3e)
fig_3f_png = plotnine_to_pil_png(fig_3f)

combine_images(columns=2, space=20, file_loc = code_folder_loc + 'output/figure3_w_age_lines.png', \
    images=[fig_3a_png, fig_3b_png, fig_3c_png, fig_3d_png, fig_3e_png, fig_3f_png])